In [62]:
import numpy as np
import pandas as pd
import emoji

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt

In [63]:
# reading train emoji and test emoji data which are attached in the repo
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [64]:
# Checking data by showing first 5 rows of the train data
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [65]:
# Checking data by showing first 5 rows of the test data
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [66]:
# Creating dictionary for some emoji's, consisting of key - number and value - emoji 
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [67]:
# Printing each emoji icon by emojizing each emoji
for ix in emoji_dict.keys():
    print (ix,end=" ")
    print (emoji.emojize(emoji_dict[ix], use_aliases=True))

0 ❤️
1 ⚾
2 😄
3 😞
4 🍴


In [68]:
# Creating training and testing data
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print ("-------------------------")
print (X_train[0],Y_train[0])

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [69]:
# Splitting the train data from sentences to words
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

# Splitting the test data from sentences to words
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()
    
# Converting labels into categorical form
Y_train = np_utils.to_categorical(Y_train)

<ipython-input-69-1de645c59088>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[ix] = X_train[ix].split()
<ipython-input-69-1de645c59088>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[ix] = X_test[ix].split()


In [70]:

# Now checking the above conversion by printing train and test data at 0th index
print (X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [71]:
# To check maximum length of sentence in training data
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4], dtype=int64))

In [72]:
# To check maximum length of senetence in testing data
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]),
 array([ 3, 12, 16, 17,  3,  4,  1], dtype=int64))

In [73]:
embeddings_index = {}

f = open('./glove.6B.50d.txt', encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [74]:
# Checking length of a particular word
embeddings_index["i"].shape

(50,)

In [75]:
from scipy import spatial
# Checking cosine similarity of words happy and sad
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.31093674898147583

In [76]:
# Filling the embedding matrix
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]

In [77]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


In [78]:
# A simple LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 10, 128)           91648     
                                                                 
 dropout_4 (Dropout)         (None, 10, 128)           0         
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 5)                 645       
                                                                 
 activation_2 (Activation)   (None, 5)                 0         
                                                                 
Total params: 223,877
Trainable params: 223,877
Non-tr

In [79]:
# Setting Loss ,Optimiser for model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
# Training model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 3s 16ms/step - loss: 1.5915 - accuracy: 0.2500
Epoch 2/50
5/5 [==============================] - 0s 17ms/step - loss: 1.5110 - accuracy: 0.3182
Epoch 3/50
5/5 [==============================] - 0s 17ms/step - loss: 1.4595 - accuracy: 0.3864
Epoch 4/50
5/5 [==============================] - 0s 17ms/step - loss: 1.4060 - accuracy: 0.3788
Epoch 5/50
5/5 [==============================] - 0s 16ms/step - loss: 1.3436 - accuracy: 0.4773
Epoch 6/50
5/5 [==============================] - 0s 21ms/step - loss: 1.2080 - accuracy: 0.5152
Epoch 7/50
5/5 [==============================] - 0s 21ms/step - loss: 1.0966 - accuracy: 0.5606
Epoch 8/50
5/5 [==============================] - 0s 19ms/step - loss: 0.9800 - accuracy: 0.6288
Epoch 9/50
5/5 [==============================] - 0s 19ms/step - loss: 0.8965 - accuracy: 0.6439
Epoch 10/50
5/5 [==============================] - 0s 28ms/step - loss: 0.7386 - accuracy: 0.7348
Epoch 11/50
5/5 [============

In [81]:
y_predict = np.argmax(model.predict(embedding_matrix_test), axis=-1)

In [82]:
# Calculating accuracy / score  of the model
float(sum(y_predict==Y_test))/embedding_matrix_test.shape[0]

0.6071428571428571

In [83]:
# Printing the sentences with the predicted and the labelled emoji
for ix in range(embedding_matrix_test.shape[0]):
    
    if y_predict[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[y_predict[ix]], use_aliases=True),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

2
['he', 'got', 'a', 'raise'] 😞 😄
3
['she', 'got', 'me', 'a', 'present'] 😄 ❤️
5
['he', 'is', 'a', 'good', 'friend'] 😄 ❤️
6
['I', 'am', 'upset'] 😞 ❤️
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 😄 ❤️
11
['work', 'is', 'hard'] 😄 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] ❤️ 😞
13
['are', 'you', 'serious', 'ha', 'ha'] 😞 😄
16
['work', 'is', 'horrible'] 😄 😞
21
['you', 'brighten', 'my', 'day'] ❤️ 😄
23
['she', 'is', 'a', 'bully'] ❤️ 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 😄 😞
28
['enjoy', 'your', 'break'] 😞 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤️
37
['I', 'am', 'starving'] 😞 🍴
40
['I', 'will', 'go', 'dance'] ⚾ 😄
41
['I', 'like', 'your', 'jacket'] ❤️ 😄
48
['I', 'want', 'to', 'joke'] 😞 😄
49
['go', 'away'] ⚾ 😞
50
['yesterday', 'we', 'lost', 'again'] ⚾ 😞
54
['You', 'totally', 'deserve', 'this', 'prize'] 😞 😄
55
['I', 'did', 'not', 'have', 'breakfast'] 🍴 😞
